In [3]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

In [4]:
import os
import math
import zipfile
import random
import collections

In [5]:
import numpy as np
from six.moves import urllib
from six.moves import xrange
import tensorflow as tf

In [14]:
url = "http://mattmahoney.net/dc/"

In [15]:
def maybe_download(filename, expected_bytes):
    """Download a file, make sure if not present and make sure it is of right size"""
    if not os.path.exists(filename):
        filename, _ = urllib.request.urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
       print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception(
           'Failed to verify ' + filename + '. can you get to it using browser')
    return filename

In [16]:
filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [18]:
#Read the data into lis of strings
def read_data(filename):
    #Extract the first file as a list of words
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

In [19]:
words = read_data(filename)
print('data size', len(words))

data size 17005207


In [20]:
#Build dictionary and replace the stopwords with UNK token
vocabulary_size = 50000

In [24]:
def build_dataset(words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size-1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

In [25]:
data, count, dictionary, reverse_dictionary = build_dataset(words)

In [26]:
del words # release memory

In [28]:
print("Most common words (+UNK)",count[:5])
print('Sample Data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample Data [5239, 3084, 12, 6, 195, 2, 3137, 46, 59, 156] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']


In [29]:
data_index = 0